In [1]:
# Imports
import pandas as pd
from sklearn import preprocessing
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2,RFE
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble._bagging import BaggingClassifier
from sklearn.ensemble._forest import ExtraTreesClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble._bagging import BaggingClassifier
from sklearn.metrics import confusion_matrix, fbeta_score

!pip install fairlearn
from fairlearn.metrics import MetricFrame
from fairlearn.metrics import selection_rate, count
from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.reductions import ExponentiatedGradient, ClassificationMoment, Moment, DemographicParity, GridSearch
from fairlearn.metrics import equalized_odds_difference, equalized_odds_ratio, false_negative_rate_difference, false_positive_rate_difference, true_positive_rate_difference, true_negative_rate_difference, demographic_parity_ratio, demographic_parity_difference

# Suppress warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.5/231.5 kB 4.3 MB/s eta 0:00:00


In [2]:
data = "/content/Teds_2019_cleaned_cri2.csv"

In [3]:
pd.set_option('display.max_columns',1000)

In [3]:
df = pd.read_csv(data)

In [5]:
df.head()

,LIVARAG_D,SUB1_D,OPSYNFLG,FREQ2_D,PSYPROB,ALCDRUG,PSOURCE,HLTHINS,DETNLF_D,MARFLG,...,RACE,FREQ1_D,PREG,FREQ2,FREQ_ATND_SELF_HELP_D,PRIMINC,DETNLF,FREQ_ATND_SELF_HELP,VET,L/S
0,3,2,0,-9,1,1,1,3,-9,0,...,1,2,2,-9,1,1,-9,1,2,1.0
1,3,2,0,2,1,3,1,3,-9,1,...,1,2,2,3,1,1,-9,1,2,1.0
2,3,2,0,2,1,3,1,2,2,1,...,8,2,2,2,1,4,2,1,2,1.0
3,3,2,0,3,1,3,1,3,-9,1,...,1,1,2,1,4,4,-9,4,2,1.0
4,3,2,0,-9,1,1,1,3,3,0,...,8,1,2,-9,1,4,3,1,2,1.0


In [ ]:

 #'LIVARAG',
 #'DIVISION',
 #'PRIMINC',
 #'AGE',
 #'EMPLOY',
 #'HLTHINS',
 #'RACE',
 #'PSYPROB',
 #'DETNLF',
 #'VET'
 #'EDUC'


In [4]:
X = df.drop(columns = ["L/S"])
y = df["L/S"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
df.shape

(1722503, 45)

In [5]:
models_to_train = [
    # LogisticRegression,
    RandomForestClassifier,
    # DecisionTreeClassifier,
    XGBClassifier,
    # BaggingClassifier
]

unfairness_metrics = [
    selection_rate,
    fbeta_score,
    equalized_odds_difference,
    equalized_odds_ratio
]
# check if demographics features in the data
demographic_features = list(set([
    'AGE',
    'EMPLOY',
    'DIVISION',
    'LIVARAG',
    'PRIMINC',
    'HLTHINS',
    'RACE',
    'PSYPROB',
    'DETNLF',
    'VET',
    'EDUC',
]).intersection(set(df.columns)))

In [6]:
# Model training
from sklearn.metrics import classification_report
def train_models_and_evaluate(X_train, y_train, X_test, y_test,
                              models=models_to_train,
                              ):
    """
    Trains models, returns evaulation metrics and also saves the results
    to a csv file in the local directory.
    Returns a list of train model instances.
    """
    df_metrics = pd.DataFrame(columns=['model', 'accuracy', 'precision', 'recall', 'f1'])
    trained_models = []
    X_train = X_train
    X_test = X_test
    for m in models:
        model = m()
        print(f"Fitting {model}...")
        model.fit(X_train, y_train)
        print(f"Predicting for {model}...")
        y_pred = model.predict(X_test)
        target_names=['0','1']
        print(classification_report(y_test, y_pred, target_names=target_names))
        # accuracy = accuracy_score(y_test, y_pred)
        # precision = precision_score(y_pred, y_test)
        # f1 = f1_score(y_pred, y_test)
        # recall = recall_score(y_pred, y_test)
        # print(str(m), accuracy, precision, recall, f1)
        df_metrics.loc[len(df_metrics.index)] = [str(m), 'accuracy', 'precision', 'recall', 'f1']
        trained_models.append(model)
    df_metrics.to_csv("models.csv", index=False)
    return trained_models

In [9]:
ms = train_models_and_evaluate(X_train, y_train, X_test, y_test)

Fitting RandomForestClassifier()...
Predicting for RandomForestClassifier()...
              precision    recall  f1-score   support

           0       0.83      0.91      0.87    360825
           1       0.73      0.57      0.64    155926

    accuracy                           0.81    516751
   macro avg       0.78      0.74      0.75    516751
weighted avg       0.80      0.81      0.80    516751

Fitting XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=Non

In [7]:
# Unfariness metrics


import sklearn.metrics as skm
import functools
from fairlearn.metrics import equalized_odds_difference, equalized_odds_ratio

fbeta_06 = functools.partial(skm.fbeta_score, beta=0.6)
fbeta_04 = functools.partial(skm.fbeta_score, beta=0.4)
fbeta_05 = functools.partial(skm.fbeta_score, beta=0.5)
def f1(y_true,y_pred):
    return skm.f1_score(y_true,y_pred)

def auc(y_true,y_pred):
    return skm.roc_auc_score(y_true,y_pred)

metric_fns = {'selection_rate': selection_rate,
              'fbeta_06': fbeta_06,
              'fbeta_04': fbeta_04,
              'fbeta_05': fbeta_05,
              'count': count,
              'f1' : f1,
              'auc' : auc}


def unfairness_metrics_evaluation(models, X_test, y_test, prefix="unmitigated"):
    """
    Saves the unfairness metrics for all models in a csv file.
    Returns a list of dictionaries with key as model object and values
    as a dictionary of unfairness metrics and their values.
    """
    df_unfairness_metrics = pd.DataFrame(columns=['categories',
                                                  'demographic_feature',
                                                  'model',
                                                  'selection_rate',
                                                  'fbeta_06',
                                                  'fbeta_04',
                                                  'fbeta_05',
                                                  'count',
                                                 'f1',
                                                 'auc'])
    df_equalized_odds = pd.DataFrame(columns=['demographic_feature',
                                              'model',
                                              'equalized_odds_difference',
                                              'equalized_odds_ratio',
                                              'demographic_parity_ratio',
                                              'demographic_parity_difference',
                                              'false_positive_rate_difference',
                                                    'false_negative_rate_difference',
                                                    'true_positive_rate_difference',
                                                     'true_negative_rate_difference',
                                                    ])

    df_overall_model_metrics = pd.DataFrame(columns=['model',
                                                    'false_positive_rate_difference',
                                                    'false_negative_rate_difference',
                                                    'true_positive_rate_difference',
                                                     'true_negative_rate_difference',
                                                    'overall_balance_error_rate'])
    for m in models:
        y_pred = m.predict(X_test)
        metrics = []
        for d in demographic_features:
            print("Model:", m)
            print("Demographic feature", d)
            results = MetricFrame(metrics=metric_fns,
                                y_true=y_test,
                                y_pred=y_pred,
                                sensitive_features=X_test[[d]])
            results = results.by_group.reset_index()
            print('result')
            results.rename({str(d): 'categories'}, inplace=True)
            results['model'] = str(m)
            print("result2")
            results['demographic_feature'] = str(d)
            print(results)
            df_unfairness_metrics = df_unfairness_metrics.append(results, ignore_index=True)
            print('result1')
            equalized_odds_diff = equalized_odds_difference(y_test, y_pred, sensitive_features=X_test[[d]])
            equalized_odds_r = equalized_odds_ratio(y_test, y_pred, sensitive_features=X_test[[d]])
            demographic_parity_r = demographic_parity_ratio(y_test, y_pred, sensitive_features=X_test[[d]])
            demographic_parity_diff = demographic_parity_difference(y_test, y_pred, sensitive_features=X_test[[d]])

            print("result4")
            false_positive_rate_diff = false_positive_rate_difference(y_test, y_pred, sensitive_features=X_test[[d]])
            false_negative_rate_diff = false_negative_rate_difference(y_test, y_pred, sensitive_features=X_test[[d]])
            true_positive_rate_diff = true_positive_rate_difference(y_test, y_pred, sensitive_features=X_test[[d]])
            true_negative_rate_diff = true_negative_rate_difference(y_test, y_pred, sensitive_features=X_test[[d]])
            # overall_balance_error = overall_balance_error_rate(y_test, y_pred, sensitive_features=X_test[d])
            print("result5")
            df_equalized_odds = df_equalized_odds.append(pd.DataFrame([[str(d), str(m),
                                                   equalized_odds_difference,
                                                   equalized_odds_ratio,
                                                   demographic_parity_ratio,
                                                   demographic_parity_difference,
                                                   false_positive_rate_diff,
                                                      false_negative_rate_diff,
                                                      true_positive_rate_diff,
                                                      true_negative_rate_diff]]),
                                     ignore_index=True)
            print("result6")

            # To do:
            # 1. Add true positive rate difference
            # 2. Add false positive rate difference

    df_equalized_odds.to_csv(f"{prefix}_Equalized_odds.csv", index=False)
    # df_overall_model_metrics.to_csv("Overall_metrics.csv", index=False)
    df_unfairness_metrics.to_csv(f"{prefix}_Unfairness_metrics.csv", index=False)
    return

In [11]:
unfairness_metrics_evaluation(ms, X_test, y_test)

Model: RandomForestClassifier()
Demographic feature EMPLOY
result
result2
   EMPLOY  selection_rate  fbeta_06  fbeta_04  fbeta_05     count        f1  \
0      -9        0.179124  0.690331  0.717422  0.703897   45717.0  0.645117   
1       1        0.311584  0.673189  0.687566  0.680451   83650.0  0.648041   
2       2        0.289804  0.670261  0.689943  0.680164   33847.0  0.636558   
3       3        0.207744  0.678115  0.705836  0.691986  190456.0  0.632044   
4       4        0.229070  0.681342  0.708121  0.694752  163081.0  0.636656   

        auc                     model demographic_feature  
0  0.753735  RandomForestClassifier()              EMPLOY  
1  0.727513  RandomForestClassifier()              EMPLOY  
2  0.722596  RandomForestClassifier()              EMPLOY  
3  0.738568  RandomForestClassifier()              EMPLOY  
4  0.736609  RandomForestClassifier()              EMPLOY  
result1
result4
result5
result6
Model: RandomForestClassifier()
Demographic feature LIVARAG

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
demographic_features

['RACE',
 'VET',
 'HLTHINS',
 'PRIMINC',
 'DIVISION',
 'LIVARAG',
 'DETNLF',
 'EDUC',
 'PSYPROB',
 'AGE',
 'EMPLOY']

In [10]:
X=df.drop(columns = ["L/S"])
y = df['L/S']
#A = df['ETHNIC']
B = df['HLTHINS']
C = df['LIVARAG']
D = df['RACE']
E = df['PSYPROB']
F = df['PRIMINC']
G = df['AGE']
H = df['EMPLOY']
I = df['DETNLF']
#J = df['MARSTAT']
#K = df['GENDER']
L = df['VET']
M = df['EDUC']
N = df['DIVISION']
X_train,X_test,y_train,y_test,B_train,B_test,C_train,C_test,D_train,D_test,E_train,E_test,F_train,F_test,G_train,G_test,H_train,H_test,I_train,I_test,L_train,L_test,M_train,M_test,N_train,N_test= train_test_split(df.drop(columns = ["L/S"]),
                                                                     df['L/S'],
                                                                     B,C,D,E,F,G,H,I,L,M,N,
                                                                     stratify=df['L/S'],
                                                                     test_size=0.3)

In [11]:
# # Fairlearn algorithms and utils
# from fairlearn.postprocessing import ThresholdOptimizer
# from fairlearn.reductions import GridSearch, EqualizedOdds

# Metrics
from fairlearn.metrics import (
    MetricFrame,
    selection_rate, demographic_parity_difference, demographic_parity_ratio,
    false_positive_rate, false_negative_rate,
    false_positive_rate_difference, false_negative_rate_difference,
    equalized_odds_difference)
from sklearn.metrics import balanced_accuracy_score, roc_auc_score

In [12]:
# Helper functions
def get_metrics_df(models_dict, y_true, group):
    metrics_dict = {
        "Overall selection rate": (
            lambda x: selection_rate(y_true, x), True),
        "Demographic parity difference": (
            lambda x: demographic_parity_difference(y_true, x, sensitive_features=group), True),
        "Demographic parity ratio": (
            lambda x: demographic_parity_ratio(y_true, x, sensitive_features=group), True),
        "------": (lambda x: "", True),
        "Overall balanced error rate": (
            lambda x: 1-balanced_accuracy_score(y_true, x), True),
        "Balanced error rate difference": (
            lambda x: MetricFrame(metrics=balanced_accuracy_score, y_true=y_true, y_pred=x, sensitive_features=group).difference(method='between_groups'), True),
        " ------": (lambda x: "", True),
        "False positive rate difference": (
            lambda x: false_positive_rate_difference(y_true, x, sensitive_features=group), True),
         " ------": (lambda x: "", True),

        "False negative rate difference": (
            lambda x: false_negative_rate_difference(y_true, x, sensitive_features=group), True),
        "Equalized odds difference": (
            lambda x: equalized_odds_difference(y_true, x, sensitive_features=group), True),
        "  ------": (lambda x: "", True),
        "Overall AUC": (
            lambda x: roc_auc_score(y_true, x), False),
        "AUC difference": (
            lambda x: MetricFrame(metrics=roc_auc_score, y_true=y_true, y_pred=x, sensitive_features=group).difference(method='between_groups'), False),
        "F1 Score:" : (
            lambda x: f1_score(y_true,x), True)
    }
    df_dict = {}
    for metric_name, (metric_func, use_preds) in metrics_dict.items():
        df_dict[metric_name] = [metric_func(preds) if use_preds else metric_func(scores)
                                for model_name, (preds, scores) in models_dict.items()]
    return pd.DataFrame.from_dict(df_dict, orient="index", columns=models_dict.keys())

XG Boost Metrics

In [13]:
#Model Training
clf = XGBClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42).fit(X_train, y_train)
clf.score(X_test, y_test)
y_pred = pd.DataFrame(clf.predict(X_test),columns=["Passed"])
target_names=['0','1']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

           0       0.77      0.89      0.83    360583
           1       0.60      0.37      0.46    156168

    accuracy                           0.74    516751
   macro avg       0.69      0.63      0.64    516751
weighted avg       0.72      0.74      0.72    516751



In [14]:
test_scores = clf.predict_proba(X_test)[:, 1]
test_preds = (test_scores >= np.mean(y_train)) * 1
# Train AUC
roc_auc_score(y_train, clf.predict_proba(X_train)[:, 1])

0.7716768305097088

HLTHINS

In [15]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=B_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
HLTHINS,,
-9,0.363802,0.219353
1,0.363538,0.289969
2,0.177001,0.470293
3,0.235890,0.354375
4,0.443695,0.196773


In [16]:
# HLTHINS Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, B_test)

,Unmitigated
Overall selection rate,0.440642
Demographic parity difference,0.320461
Demographic parity ratio,0.44773
------,
Overall balanced error rate,0.294375
Balanced error rate difference,0.035176
------,
False positive rate difference,0.266695
False negative rate difference,0.27352
Equalized odds difference,0.27352


LIVARAG

In [17]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=C_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
LIVARAG,,
-9,0.082966,0.711267
1,0.183702,0.380936
2,0.475944,0.192272
3,0.364336,0.233167


In [18]:
# LIVARAG Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, C_test)

,Unmitigated
Overall selection rate,0.440642
Demographic parity difference,0.483626
Demographic parity ratio,0.203529
------,
Overall balanced error rate,0.294375
Balanced error rate difference,0.114797
------,
False positive rate difference,0.392978
False negative rate difference,0.518995
Equalized odds difference,0.518995


RACE

In [19]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=D_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
RACE,,
-9,0.117672,0.582573
1,0.370370,0.221311
2,0.229408,0.317524
3,0.184466,0.422222
4,0.330683,0.270938
5,0.311824,0.276621
6,0.426667,0.199330
7,0.427552,0.168466
8,0.423301,0.233274


In [20]:
# RACE Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, D_test)

,Unmitigated
Overall selection rate,0.440642
Demographic parity difference,0.393138
Demographic parity ratio,0.30871
------,
Overall balanced error rate,0.294375
Balanced error rate difference,0.076656
------,
False positive rate difference,0.30988
False negative rate difference,0.414107
Equalized odds difference,0.414107


PSYPROB

In [21]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=E_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
PSYPROB,,
-9,0.318908,0.163554
1,0.279010,0.327324
2,0.347249,0.258455


In [22]:
# PSYPROB Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, E_test)

,Unmitigated
Overall selection rate,0.440642
Demographic parity difference,0.078779
Demographic parity ratio,0.833056
------,
Overall balanced error rate,0.294375
Balanced error rate difference,0.061936
------,
False positive rate difference,0.06824
False negative rate difference,0.16377
Equalized odds difference,0.16377


PRIMINC

In [23]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=F_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
PRIMINC,,
-9,0.289092,0.297386
1,0.453360,0.171567
2,0.295352,0.298902
3,0.290095,0.295481
4,0.334454,0.239007
5,0.267465,0.334916


In [24]:
# PRIMINC Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, F_test)

,Unmitigated
Overall selection rate,0.440642
Demographic parity difference,0.21237
Demographic parity ratio,0.638057
------,
Overall balanced error rate,0.294375
Balanced error rate difference,0.025733
------,
False positive rate difference,0.185894
False negative rate difference,0.163349
Equalized odds difference,0.185894


AGE

In [25]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=G_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
AGE,,
1,0.824758,0.107765
2,0.789085,0.105400
3,0.467896,0.229123
4,0.387952,0.243482
5,0.331485,0.273172
6,0.308285,0.286832
7,0.297392,0.288104
8,0.274814,0.296868
9,0.284329,0.279895


In [26]:
# AGE Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, G_test)

,Unmitigated
Overall selection rate,0.440642
Demographic parity difference,0.47387
Demographic parity ratio,0.447363
------,
Overall balanced error rate,0.294375
Balanced error rate difference,0.191527
------,
False positive rate difference,0.565005
False negative rate difference,0.191468
Equalized odds difference,0.565005


EMPLOY

In [27]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=H_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
EMPLOY,,
-9,0.123124,0.520458
1,0.494826,0.166759
2,0.475772,0.192108
3,0.270752,0.293556
4,0.317400,0.281230


In [28]:
# EMPLOY Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, H_test)

,Unmitigated
Overall selection rate,0.440642
Demographic parity difference,0.411831
Demographic parity ratio,0.334685
------,
Overall balanced error rate,0.294375
Balanced error rate difference,0.051786
------,
False positive rate difference,0.371702
False negative rate difference,0.353699
Equalized odds difference,0.371702


DETNLF

In [29]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=I_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
DETNLF,,
-9,0.330260,0.255104
1,0.296542,0.253456
2,0.645843,0.147303
3,0.324374,0.282083
4,0.511828,0.244786
5,0.211749,0.402880


In [30]:
# DETNLF Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, I_test)

,Unmitigated
Overall selection rate,0.440642
Demographic parity difference,0.432297
Demographic parity ratio,0.413765
------,
Overall balanced error rate,0.294375
Balanced error rate difference,0.121574
------,
False positive rate difference,0.434094
False negative rate difference,0.255577
Equalized odds difference,0.434094


VET

In [31]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=L_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
VET,,
-9,0.172834,0.417365
1,0.358095,0.267748
2,0.332834,0.258556


In [32]:
# VET Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, L_test)

,Unmitigated
Overall selection rate,0.440642
Demographic parity difference,0.203982
Demographic parity ratio,0.577172
------,
Overall balanced error rate,0.294375
Balanced error rate difference,0.017822
------,
False positive rate difference,0.185261
False negative rate difference,0.158809
Equalized odds difference,0.185261


EDUC

In [33]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=M_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
EDUC,,
-9,0.086248,0.712650
1,0.339290,0.234028
2,0.388966,0.222486
3,0.332426,0.250574
4,0.335923,0.243701
5,0.366446,0.249207


In [34]:
# EDUC Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, M_test)

,Unmitigated
Overall selection rate,0.440642
Demographic parity difference,0.396328
Demographic parity ratio,0.240044
------,
Overall balanced error rate,0.294375
Balanced error rate difference,0.11279
------,
False positive rate difference,0.302718
False negative rate difference,0.490164
Equalized odds difference,0.490164


DIVISION

In [35]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=N_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
DIVISION,,
0,0.610548,0.172515
1,0.273154,0.380688
2,0.364337,0.214126
3,0.371773,0.324700
4,0.532889,0.238251
5,0.245137,0.267884
6,0.154692,0.377276
7,0.432251,0.298937
8,0.116041,0.485646


In [36]:
# DIVISION Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, N_test)

,Unmitigated
Overall selection rate,0.440642
Demographic parity difference,0.552969
Demographic parity ratio,0.267553
------,
Overall balanced error rate,0.294375
Balanced error rate difference,0.135021
------,
False positive rate difference,0.531273
False negative rate difference,0.378046
Equalized odds difference,0.531273


Exponentiated Gradient algorithm

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
from fairlearn.metrics import selection_rate, demographic_parity_ratio
from fairlearn.reductions import ExponentiatedGradient, DemographicParity
from sklearn.ensemble import RandomForestClassifier


In [38]:
# Train a machine learning model
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

DIVISION

In [45]:
sensitive_attribute = N_train
constraint = DemographicParity()

mitigator = ExponentiatedGradient(
    estimator=XGBClassifier(),
    constraints=constraint
)
mitigator.fit(X_train, y_train, sensitive_features=sensitive_attribute)

sensitive_attribute = N_test
# Apply the mitigation to the predictions
final_predictions = mitigator.predict(X_test)

In [47]:
# DIVISION Metrics
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ExponentiatedGradient": (final_predictions, final_predictions)}
get_metrics_df(models_dict, y_test, N_test)

,Unmitigated,ExponentiatedGradient
Overall selection rate,0.442401,0.227984
Demographic parity difference,0.599824,0.032608
Demographic parity ratio,0.265573,0.868209
------,,
Overall balanced error rate,0.295181,0.280209
Balanced error rate difference,0.186287,0.149425
------,,
False positive rate difference,0.639053,0.076308
False negative rate difference,0.364111,0.3063
Equalized odds difference,0.639053,0.3063


EDUC

In [79]:
sensitive_attribute = M_train
constraint = DemographicParity()

mitigator = ExponentiatedGradient(
    estimator=XGBClassifier(),
    constraints=constraint
)
mitigator.fit(X_train, y_train, sensitive_features=sensitive_attribute)

sensitive_attribute = M_test
# Apply the mitigation to the predictions
final_predictions = mitigator.predict(X_test)

In [84]:
# EDUC Metrics
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ExponentiatedGradient": (final_predictions, final_predictions)}
get_metrics_df(models_dict, y_test, M_test)

,Unmitigated,ExponentiatedGradient
Overall selection rate,0.442401,0.229157
Demographic parity difference,0.396746,0.016803
Demographic parity ratio,0.239361,0.929485
------,,
Overall balanced error rate,0.295181,0.275002
Balanced error rate difference,0.113111,0.035559
------,,
False positive rate difference,0.305654,0.037559
False negative rate difference,0.488639,0.106697
Equalized odds difference,0.488639,0.106697


VET

In [85]:
sensitive_attribute = L_train
constraint = DemographicParity()

mitigator = ExponentiatedGradient(
    estimator=XGBClassifier(),
    constraints=constraint
)
mitigator.fit(X_train, y_train, sensitive_features=sensitive_attribute)

sensitive_attribute = L_test
# Apply the mitigation to the predictions
final_predictions = mitigator.predict(X_test)

In [86]:
# VET Metrics
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ExponentiatedGradient": (final_predictions, final_predictions)}
get_metrics_df(models_dict, y_test, M_test)

,Unmitigated,ExponentiatedGradient
Overall selection rate,0.442401,0.238271
Demographic parity difference,0.396746,0.155108
Demographic parity ratio,0.239361,0.441923
------,,
Overall balanced error rate,0.295181,0.266442
Balanced error rate difference,0.113111,0.046806
------,,
False positive rate difference,0.305654,0.072773
False negative rate difference,0.488639,0.152257
Equalized odds difference,0.488639,0.152257


HLTHINS

In [87]:
sensitive_attribute = B_train
constraint = DemographicParity()

mitigator = ExponentiatedGradient(
    estimator=XGBClassifier(),
    constraints=constraint
)
mitigator.fit(X_train, y_train, sensitive_features=sensitive_attribute)

sensitive_attribute = B_test
# Apply the mitigation to the predictions
final_predictions = mitigator.predict(X_test)

In [88]:
# HLTHINS Metrics
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ExponentiatedGradient": (final_predictions, final_predictions)}
get_metrics_df(models_dict, y_test, B_test)

,Unmitigated,ExponentiatedGradient
Overall selection rate,0.442401,0.231179
Demographic parity difference,0.310842,0.020438
Demographic parity ratio,0.460945,0.915659
------,,
Overall balanced error rate,0.295181,0.274111
Balanced error rate difference,0.035348,0.032136
------,,
False positive rate difference,0.256208,0.024364
False negative rate difference,0.269479,0.082327
Equalized odds difference,0.269479,0.082327


LIVARAG

In [89]:
sensitive_attribute = C_train
constraint = DemographicParity()

mitigator = ExponentiatedGradient(
    estimator=XGBClassifier(),
    constraints=constraint
)
mitigator.fit(X_train, y_train, sensitive_features=sensitive_attribute)

sensitive_attribute = C_test
# Apply the mitigation to the predictions
final_predictions = mitigator.predict(X_test)

In [90]:
# LIVARAG Metrics
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ExponentiatedGradient": (final_predictions, final_predictions)}
get_metrics_df(models_dict, y_test, C_test)

,Unmitigated,ExponentiatedGradient
Overall selection rate,0.442401,0.224298
Demographic parity difference,0.490028,0.021788
Demographic parity ratio,0.199204,0.907606
------,,
Overall balanced error rate,0.295181,0.280445
Balanced error rate difference,0.112401,0.071119
------,,
False positive rate difference,0.403625,0.039588
False negative rate difference,0.516632,0.159236
Equalized odds difference,0.516632,0.159236


RACE

In [91]:
sensitive_attribute = D_train
constraint = DemographicParity()

mitigator = ExponentiatedGradient(
    estimator=XGBClassifier(),
    constraints=constraint
)
mitigator.fit(X_train, y_train, sensitive_features=sensitive_attribute)

sensitive_attribute = D_test
# Apply the mitigation to the predictions
final_predictions = mitigator.predict(X_test)

In [92]:
# RACE Metrics
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ExponentiatedGradient": (final_predictions, final_predictions)}
get_metrics_df(models_dict, y_test, D_test)

,Unmitigated,ExponentiatedGradient
Overall selection rate,0.442401,0.236479
Demographic parity difference,0.395628,0.09416
Demographic parity ratio,0.31748,0.709674
------,,
Overall balanced error rate,0.295181,0.270655
Balanced error rate difference,0.086956,0.105502
------,,
False positive rate difference,0.320793,0.096074
False negative rate difference,0.408841,0.264814
Equalized odds difference,0.408841,0.264814


PSYPROB

In [39]:
sensitive_attribute = E_train
constraint = DemographicParity()

mitigator = ExponentiatedGradient(
    estimator=XGBClassifier(),
    constraints=constraint
)
mitigator.fit(X_train, y_train, sensitive_features=sensitive_attribute)

sensitive_attribute = E_test
# Apply the mitigation to the predictions
final_predictions = mitigator.predict(X_test)

In [40]:
# PSYPROB Metrics
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ExponentiatedGradient": (final_predictions, final_predictions)}
get_metrics_df(models_dict, y_test, E_test)

,Unmitigated,ExponentiatedGradient
Overall selection rate,0.440642,0.235601
Demographic parity difference,0.078779,0.02228
Demographic parity ratio,0.833056,0.909886
------,,
Overall balanced error rate,0.294375,0.267745
Balanced error rate difference,0.061936,0.011794
------,,
False positive rate difference,0.06824,0.018104
False negative rate difference,0.16377,0.035744
Equalized odds difference,0.16377,0.035744


PRIMINC

In [41]:
sensitive_attribute = F_train
constraint = DemographicParity()

mitigator = ExponentiatedGradient(
    estimator=XGBClassifier(),
    constraints=constraint
)
mitigator.fit(X_train, y_train, sensitive_features=sensitive_attribute)

sensitive_attribute = F_test
# Apply the mitigation to the predictions
final_predictions = mitigator.predict(X_test)

In [42]:
# PRIMINC Metrics
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ExponentiatedGradient": (final_predictions, final_predictions)}
get_metrics_df(models_dict, y_test, F_test)

,Unmitigated,ExponentiatedGradient
Overall selection rate,0.440642,0.234177
Demographic parity difference,0.21237,0.019556
Demographic parity ratio,0.638057,0.920469
------,,
Overall balanced error rate,0.294375,0.269854
Balanced error rate difference,0.025733,0.044667
------,,
False positive rate difference,0.185894,0.016413
False negative rate difference,0.163349,0.072921
Equalized odds difference,0.185894,0.072921


AGE

In [43]:
sensitive_attribute = G_train
constraint = DemographicParity()

mitigator = ExponentiatedGradient(
    estimator=XGBClassifier(),
    constraints=constraint
)
mitigator.fit(X_train, y_train, sensitive_features=sensitive_attribute)

sensitive_attribute = G_test
# Apply the mitigation to the predictions
final_predictions = mitigator.predict(X_test)

In [44]:
# AGE Metrics
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ExponentiatedGradient": (final_predictions, final_predictions)}
get_metrics_df(models_dict, y_test, G_test)

,Unmitigated,ExponentiatedGradient
Overall selection rate,0.440642,0.235111
Demographic parity difference,0.47387,0.020593
Demographic parity ratio,0.447363,0.916407
------,,
Overall balanced error rate,0.294375,0.270386
Balanced error rate difference,0.191527,0.111993
------,,
False positive rate difference,0.565005,0.029187
False negative rate difference,0.191468,0.207137
Equalized odds difference,0.565005,0.207137


EMPLOY

In [45]:
sensitive_attribute = H_train
constraint = DemographicParity()

mitigator = ExponentiatedGradient(
    estimator=XGBClassifier(),
    constraints=constraint
)
mitigator.fit(X_train, y_train, sensitive_features=sensitive_attribute)

sensitive_attribute = H_test
# Apply the mitigation to the predictions
final_predictions = mitigator.predict(X_test)

In [46]:
# EMPLOY Metrics
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ExponentiatedGradient": (final_predictions, final_predictions)}
get_metrics_df(models_dict, y_test, H_test)

,Unmitigated,ExponentiatedGradient
Overall selection rate,0.440642,0.230608
Demographic parity difference,0.411831,0.023728
Demographic parity ratio,0.334685,0.901975
------,,
Overall balanced error rate,0.294375,0.271399
Balanced error rate difference,0.051786,0.085517
------,,
False positive rate difference,0.371702,0.012866
False negative rate difference,0.353699,0.158167
Equalized odds difference,0.371702,0.158167


DETNLF

In [47]:
sensitive_attribute = I_train
constraint = DemographicParity()

mitigator = ExponentiatedGradient(
    estimator=XGBClassifier(),
    constraints=constraint
)
mitigator.fit(X_train, y_train, sensitive_features=sensitive_attribute)

sensitive_attribute = I_test
# Apply the mitigation to the predictions
final_predictions = mitigator.predict(X_test)

In [48]:
# DETNLF Metrics
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ExponentiatedGradient": (final_predictions, final_predictions)}
get_metrics_df(models_dict, y_test, I_test)

,Unmitigated,ExponentiatedGradient
Overall selection rate,0.440642,0.232925
Demographic parity difference,0.432297,0.030705
Demographic parity ratio,0.413765,0.878813
------,,
Overall balanced error rate,0.294375,0.271177
Balanced error rate difference,0.121574,0.091547
------,,
False positive rate difference,0.434094,0.028691
False negative rate difference,0.255577,0.185068
Equalized odds difference,0.434094,0.185068


CorrelationRemover

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from fairlearn.datasets import fetch_diabetes_hospital
from fairlearn.preprocessing import CorrelationRemover

In [49]:
correlation_threshold = 0.5  # Adjust this threshold as needed

# Calculate the correlation matrix
correlation_matrix = df.corr()

In [50]:
correlation_matrix

,LIVARAG_D,SUB1_D,OPSYNFLG,FREQ2_D,PSYPROB,ALCDRUG,PSOURCE,HLTHINS,DETNLF_D,MARFLG,CBSA2010,SUB2,ROUTE1,REASON,BENZFLG,PRIMPAY,EMPLOY_D,FRSTUSE1,SUB3,FREQ3,SUB1,DSMCRIT,FREQ1,LIVARAG,MTHAMFLG,METHUSE,NOPRIOR,DAYWAIT,EDUC,EMPLOY,REGION,AGE,STFIPS,DETCRIM,DIVISION,RACE,FREQ1_D,PREG,FREQ2,FREQ_ATND_SELF_HELP_D,PRIMINC,DETNLF,FREQ_ATND_SELF_HELP,VET,L/S
LIVARAG_D,1.000000,0.495294,0.091244,0.234325,0.172201,0.243872,0.371478,-0.035144,0.157833,0.095648,0.113459,0.037610,0.364491,0.206890,0.027040,-0.088969,0.774447,0.347005,0.158413,0.076010,0.305324,0.022274,0.277403,0.598088,-0.018640,-0.101981,0.302448,0.121464,0.507657,0.401738,-0.290637,-0.041983,0.306655,0.134248,-0.339420,0.196511,0.553957,0.031363,0.105244,0.462274,0.134757,0.027871,0.426400,0.395572,0.087337
SUB1_D,0.495294,1.000000,0.165694,0.265362,0.167827,0.308980,0.365129,-0.051069,0.109568,0.102009,0.066378,0.076850,0.441188,0.243122,0.060222,-0.105725,0.535049,0.398980,-0.050504,0.077589,0.665860,-0.163477,0.288354,0.421407,0.353489,0.019524,0.311570,0.199606,0.407860,0.386567,0.034589,-0.144920,0.117551,0.117400,0.013507,0.203328,0.516807,0.094633,0.159866,0.256205,-0.018835,-0.065009,0.478084,0.372199,0.105003
OPSYNFLG,0.091244,0.165694,1.000000,-0.025024,0.021732,0.116731,0.024389,-0.024284,-0.015673,-0.049244,-0.008664,0.169124,0.102704,0.113998,0.076708,-0.023686,0.071063,0.154460,0.140523,0.197809,0.206912,0.037355,0.098764,0.085115,-0.016863,0.028195,0.035592,-0.049549,0.072362,0.058879,-0.011186,-0.048388,0.095083,-0.033617,-0.035441,0.037092,0.095795,0.064154,0.173412,0.034330,-0.035158,-0.024796,0.053659,0.058836,0.004997
FREQ2_D,0.234325,0.265362,-0.025024,1.000000,0.049038,0.432368,0.095509,-0.069299,0.079407,0.335815,0.059659,0.034196,0.170588,0.105515,-0.034222,-0.096422,0.244972,0.147516,0.136373,0.265334,0.155479,0.020907,0.098713,0.137494,0.020601,0.059714,0.108592,-0.031725,0.136966,0.126522,-0.098748,-0.031827,0.104006,0.002103,-0.111429,0.056107,0.242625,-0.002100,0.530179,0.173709,0.019747,0.026377,0.189785,0.114769,0.027173
PSYPROB,0.172201,0.167827,0.021732,0.049038,1.000000,-0.073933,-0.065050,0.160863,0.080895,-0.022533,-0.077985,-0.019929,-0.070050,0.060942,-0.041892,0.092225,0.163977,-0.036637,-0.103800,-0.057617,-0.053523,0.087223,-0.113956,-0.100368,-0.000521,-0.121787,0.391851,0.165618,-0.106777,-0.099052,0.151255,-0.019252,-0.281134,0.083760,0.104570,-0.044361,0.043999,0.062565,-0.050440,-0.138148,-0.161164,-0.076882,0.162798,0.089143,0.014669
ALCDRUG,0.243872,0.308980,0.116731,0.432368,-0.073933,1.000000,0.379598,-0.108670,0.060141,0.358000,0.156137,0.319676,0.616591,0.118578,0.094794,-0.151388,0.221265,0.557348,0.081086,0.354430,0.502969,0.009694,0.519034,0.410503,0.175839,0.034503,-0.003744,0.110221,0.402875,0.433548,-0.212139,-0.082208,0.209230,0.073003,-0.194418,0.182979,0.222482,-0.033770,0.671370,0.217491,0.123288,0.111069,0.260038,0.399492,0.068432
PSOURCE,0.371478,0.365129,0.024389,0.095509,-0.065050,0.379598,1.000000,-0.081495,0.032243,0.216180,0.146196,0.094288,0.495060,0.120822,0.007205,-0.071931,0.325304,0.454359,0.002828,0.073794,0.485988,-0.050345,0.391044,0.533492,0.155688,-0.026109,0.054746,0.216878,0.510844,0.523564,-0.190740,-0.108644,0.258909,0.529543,-0.161183,0.219792,0.207982,-0.019905,0.163697,0.270780,0.183131,0.052197,0.375060,0.477969,0.150022
HLTHINS,-0.035144,-0.051069,-0.024284,-0.069299,0.160863,-0.108670,-0.081495,1.000000,0.058663,-0.003995,-0.217713,-0.030666,-0.139203,-0.106469,-0.001660,0.751234,0.019189,-0.165308,0.140915,0.021480,-0.105406,-0.055230,-0.050373,-0.247418,0.084955,-0.036695,0.118596,0.352852,-0.247294,-0.236695,0.186124,-0.031685,-0.136176,0.182735,0.181397,-0.159855,0.009846,0.044628,-0.050202,-0.206766,0.379028,-0.000036,-0.142036,-0.115790,-0.023081
DETNLF_D,0.157833,0.109568,-0.015673,0.079407,0.080895,0.060141,0.032243,0.058663,1.000000,0.009793,0.019641,0.008560,0.077806,0.015383,0.037609,0.034239,0.388698,0.064238,0.100682,0.038075,0.035451,0.084631,0.053677,0.039082,-0.069523

In [51]:
df.shape

(1722503, 45)

In [52]:
features_to_remove = set()

In [53]:
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > correlation_threshold:
            colname = correlation_matrix.columns[i]
            features_to_remove.add(colname)

# Remove highly correlated features from the DataFrame
data_cleaned = df.drop(columns=features_to_remove)

In [54]:
data_cleaned.shape

(1722503, 28)

In [55]:
X1 = data_cleaned.drop(columns = ["L/S"])
y1 = data_cleaned["L/S"]
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3, random_state=42)

In [61]:
X1=data_cleaned.drop(columns = ["L/S"])
y1 = data_cleaned['L/S']
#A = df['ETHNIC']
B1 = data_cleaned['HLTHINS']
#C1 = data_cleaned['LIVARAG']
D1 = data_cleaned['RACE']
E1 = data_cleaned['PSYPROB']
F1 = data_cleaned['PRIMINC']
G1 = data_cleaned['AGE']
#H1 = data_cleaned['EMPLOY']
#I1 = data_cleaned['DETNLF']
#J = df['MARSTAT']
#K = df['GENDER']
#L1 = data_cleaned['VET']
#M1 = data_cleaned['EDUC']
#N1 = data_cleaned['DIVISION']
X1_train,X1_test,y1_train,y1_test,B1_train,B1_test,D1_train,D1_test,E1_train,E1_test,F1_train,F1_test,G1_train,G1_test= train_test_split(data_cleaned.drop(columns = ["L/S"]),
                                                                     data_cleaned['L/S'],
                                                                     B1,D1,E1,F1,G1,
                                                                     stratify=df['L/S'],
                                                                     test_size=0.3)

In [62]:
#Model Training
clf1 = XGBClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42).fit(X1_train, y1_train)
clf1.score(X1_test, y1_test)
y1_pred = pd.DataFrame(clf1.predict(X1_test),columns=["Passed"])
target_names=['0','1']
print(classification_report(y1_test, y1_pred, target_names=target_names))

              precision    recall  f1-score   support

           0       0.75      0.91      0.82    360583
           1       0.59      0.30      0.40    156168

    accuracy                           0.72    516751
   macro avg       0.67      0.60      0.61    516751
weighted avg       0.70      0.72      0.69    516751



In [63]:
cr_scores = clf1.predict_proba(X1_test)[:, 1]
cr_preds = (cr_scores >= np.mean(y1_train)) * 1
# Train AUC
roc_auc_score(y1_train, clf1.predict_proba(X1_train)[:, 1])

0.7487567039161113

HLTHINS

In [64]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y1_test, cr_preds, sensitive_features=B1_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
HLTHINS,,
-9,0.398152,0.228743
1,0.365443,0.320810
2,0.200860,0.473078
3,0.258419,0.381173
4,0.516370,0.199857


In [65]:
# HLTHINS Metrics
models_dict = {"CorrelationRemover": (cr_preds, cr_scores)}
get_metrics_df(models_dict, y1_test, B1_test)

,CorrelationRemover
Overall selection rate,0.462416
Demographic parity difference,0.346369
Demographic parity ratio,0.444775
------,
Overall balanced error rate,0.316002
Balanced error rate difference,0.044666
------,
False positive rate difference,0.31551
False negative rate difference,0.273221
Equalized odds difference,0.31551


In [66]:
# HLTHINS Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, B_test)

,Unmitigated
Overall selection rate,0.442401
Demographic parity difference,0.310842
Demographic parity ratio,0.460945
------,
Overall balanced error rate,0.295181
Balanced error rate difference,0.035348
------,
False positive rate difference,0.256208
False negative rate difference,0.269479
Equalized odds difference,0.269479


RACE

In [67]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y1_test, cr_preds, sensitive_features=D1_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
RACE,,
-9,0.108663,0.649138
1,0.444109,0.155102
2,0.256133,0.368827
3,0.202128,0.531250
4,0.372937,0.267730
5,0.346986,0.286919
6,0.470015,0.216216
7,0.463245,0.161629
8,0.485367,0.233600


In [68]:
# RACE Metrics
models_dict = {"CorrelationRemover": (cr_preds, cr_scores)}
get_metrics_df(models_dict, y1_test, D1_test)

,CorrelationRemover
Overall selection rate,0.462416
Demographic parity difference,0.458981
Demographic parity ratio,0.253184
------,
Overall balanced error rate,0.316002
Balanced error rate difference,0.092577
------,
False positive rate difference,0.376704
False negative rate difference,0.494036
Equalized odds difference,0.494036


In [69]:
# RACE Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, D_test)

,Unmitigated
Overall selection rate,0.442401
Demographic parity difference,0.395628
Demographic parity ratio,0.31748
------,
Overall balanced error rate,0.295181
Balanced error rate difference,0.086956
------,
False positive rate difference,0.320793
False negative rate difference,0.408841
Equalized odds difference,0.408841


PSYPROB

In [70]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y1_test, cr_preds, sensitive_features=E1_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
PSYPROB,,
-9,0.351420,0.174610
1,0.307582,0.338232
2,0.388058,0.264524


In [71]:
# PSYPROB Metrics
models_dict = {"CorrelationRemover": (cr_preds, cr_scores)}
get_metrics_df(models_dict, y1_test, E1_test)

,CorrelationRemover
Overall selection rate,0.462416
Demographic parity difference,0.087904
Demographic parity ratio,0.823481
------,
Overall balanced error rate,0.316002
Balanced error rate difference,0.063276
------,
False positive rate difference,0.080476
False negative rate difference,0.163622
Equalized odds difference,0.163622


In [72]:
# PSYPROB Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, E_test)

,Unmitigated
Overall selection rate,0.442401
Demographic parity difference,0.078139
Demographic parity ratio,0.835489
------,
Overall balanced error rate,0.295181
Balanced error rate difference,0.061377
------,
False positive rate difference,0.066972
False negative rate difference,0.149799
Equalized odds difference,0.149799


PRIMINC

In [73]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y1_test, cr_preds, sensitive_features=F1_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
PRIMINC,,
-9,0.321789,0.302254
1,0.499885,0.169673
2,0.360121,0.273542
3,0.350051,0.278035
4,0.348071,0.261104
5,0.296312,0.368345


In [74]:
# PRIMINC Metrics
models_dict = {"CorrelationRemover": (cr_preds, cr_scores)}
get_metrics_df(models_dict, y1_test, F1_test)

,CorrelationRemover
Overall selection rate,0.462416
Demographic parity difference,0.230842
Demographic parity ratio,0.625837
------,
Overall balanced error rate,0.316002
Balanced error rate difference,0.030191
------,
False positive rate difference,0.203572
False negative rate difference,0.198672
Equalized odds difference,0.203572


In [75]:
# PRIMINC Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, F_test)

,Unmitigated
Overall selection rate,0.442401
Demographic parity difference,0.210771
Demographic parity ratio,0.643877
------,
Overall balanced error rate,0.295181
Balanced error rate difference,0.02609
------,
False positive rate difference,0.185464
False negative rate difference,0.158659
Equalized odds difference,0.185464


AGE

In [76]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y1_test, cr_preds, sensitive_features=G1_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
AGE,,
1,0.810646,0.112157
2,0.805778,0.080264
3,0.505679,0.236676
4,0.419927,0.244222
5,0.373312,0.274853
6,0.354737,0.283941
7,0.342581,0.285045
8,0.321829,0.293522
9,0.298090,0.316804


In [77]:
# AGE Metrics
models_dict = {"CorrelationRemover": (cr_preds, cr_scores)}
get_metrics_df(models_dict, y1_test, G1_test)

,CorrelationRemover
Overall selection rate,0.462416
Demographic parity difference,0.473251
Demographic parity ratio,0.449232
------,
Overall balanced error rate,0.316002
Balanced error rate difference,0.156151
------,
False positive rate difference,0.530939
False negative rate difference,0.250809
Equalized odds difference,0.530939


In [78]:
# AGE Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, G_test)

,Unmitigated
Overall selection rate,0.442401
Demographic parity difference,0.45243
Demographic parity ratio,0.459235
------,
Overall balanced error rate,0.295181
Balanced error rate difference,0.172682
------,
False positive rate difference,0.527804
False negative rate difference,0.19127
Equalized odds difference,0.527804
